# Aggregate Functions & GroupBy

In [1]:
import findspark
findspark.init('/home/aforestier10/Downloads/spark-3.5.3-bin-hadoop3')

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('Agg').getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

## Read in data

In [8]:
df = spark.read.csv('sales_info.csv', inferSchema=True, header=True)
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [9]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



### I want float for sales, let's do that

In [10]:
from pyspark.sql.types import (StructField, StringType, FloatType, StructType)

In [13]:
data_schema = [StructField('Company', StringType(), True),
              StructField('Person', StringType(), True),
              StructField('Sales', FloatType(), True)]

final_struct = StructType(fields=data_schema)

In [19]:
df = spark.read.csv('sales_info.csv', final_struct)
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: float (nullable = true)



## Groupby

In [20]:
df.groupBy("Company")

GroupedData[grouping expressions: [Company], value: [Company: string, Person: string ... 1 more field], type: GroupBy]

In [21]:
df.groupBy("Company").mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|Company|             NULL|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [22]:
df.groupBy("Company").min().show()

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|Company|      NULL|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



In [23]:
df.groupBy("Company").count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|Company|    1|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



## Aggregate functions

In [24]:
df.agg({'Sales': 'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [25]:
df.agg({'Sales': 'max'}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



## Combined for more generalized approach

In [28]:
grouped = df.groupBy('Person')
grouped.agg({'Sales': 'Max'}).show()

+-------+----------+
| Person|max(Sales)|
+-------+----------+
|  Linda|     130.0|
| Person|      NULL|
|    Amy|     124.0|
|  Sarah|     350.0|
|Charlie|     120.0|
|  Chris|     350.0|
|   John|     250.0|
|    Sam|     200.0|
|   Carl|     870.0|
|Vanessa|     243.0|
|   Mike|     750.0|
|   Tina|     600.0|
|  Frank|     340.0|
+-------+----------+



## Spark Functions

In [29]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [34]:
df.select(countDistinct('Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [35]:
df.select(avg('Sales').alias('Avg Sales')).show()

+-----------------+
|        Avg Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [36]:
df.select(stddev('Sales')).show()

+------------------+
|     stddev(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [39]:
# Format 
from pyspark.sql.functions import format_number
sales_std = df.select(stddev('Sales').alias('Sales std'))
sales_std.select(format_number('Sales std',2).alias('Sales std')).show()

+---------+
|Sales std|
+---------+
|   250.09|
+---------+



### Order

In [40]:
df.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|Company| Person| NULL|
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [42]:
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
|Company| Person| NULL|
+-------+-------+-----+

